In [1]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.advanced_activations import PReLU
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
def show_train_history(train_history, train, validation):
    plt.plot(train_history.history[train])
    plt.plot(train_history.history[validation])
    plt.title('Train History')
    plt.ylabel('train')
    plt.xlabel('Epoch')
    plt.legend(['train', 'validation'], loc='center right')
    plt.show()

In [3]:
from keras.callbacks import ReduceLROnPlateau
learning_rate_function = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [4]:
# CNN
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(1, activation='sigmoid'))
# compile model
opt = SGD(lr=1e-4, momentum=0.1)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 200, 200, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 100, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 50, 50, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 25, 25, 128)       0         
_________________________________________________________________
flat

In [6]:
BATCH_SIZE = 64
train_datagen = ImageDataGenerator(featurewise_center=True)
valid_datagen = ImageDataGenerator(featurewise_center=True)

# specify imagenet mean values for centering
train_datagen.mean = [123.68, 116.779, 103.939]
valid_datagen.mean = [123.68, 116.779, 103.939]

train_generator = train_datagen.flow_from_directory(
        './train',
        target_size=(200, 200),
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False
    )

validation_generator = test_datagen.flow_from_directory(
        './valid',
        target_size=(200, 200),
        batch_size=BATCH_SIZE,
        class_mode='binary'
    )

train_history = model.fit_generator(
        train_generator,
        epochs=30,
        steps_per_epoch = len(train_generator),
        validation_data = validation_generator,
        validation_steps = len(validation_generator),
        verbose = 2,
        callbacks=[learning_rate_function]
)
# show_train_history
show_train_history(train_history, 'acc', 'val_acc')
show_train_history(train_history, 'loss', 'val_loss')
# save model
model.save('model-CNN.h5')

Found 3200 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Epoch 1/30
 - 14s - loss: 0.6897 - acc: 0.9569 - val_loss: 0.7021 - val_acc: 0.5337
Epoch 2/30
 - 13s - loss: 10.1629 - acc: 0.3672 - val_loss: 2.7289 - val_acc: 0.5000
Epoch 3/30
 - 13s - loss: 8.0590 - acc: 0.5000 - val_loss: 2.7289 - val_acc: 0.5000
Epoch 4/30
 - 13s - loss: 8.0590 - acc: 0.5000 - val_loss: 2.7289 - val_acc: 0.5000

Epoch 00004: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
Epoch 5/30
 - 13s - loss: 8.0590 - acc: 0.5000 - val_loss: 2.7289 - val_acc: 0.5000
Epoch 6/30
 - 13s - loss: 8.0590 - acc: 0.5000 - val_loss: 2.7289 - val_acc: 0.5000
Epoch 7/30
 - 13s - loss: 8.0590 - acc: 0.5000 - val_loss: 2.7289 - val_acc: 0.5000

Epoch 00007: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-05.
Epoch 8/30


KeyboardInterrupt: 

In [3]:
# VGG16
from keras.applications.vgg16 import VGG16, decode_predictions

# load model
model = VGG16(include_top=False, input_shape=(224, 224, 3))
# mark loaded layers as not trainable
for layer in model.layers:
    layer.trainable = False
# add new classifier layers
flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
output = Dense(1, activation='sigmoid')(class1)
# define new model
model = Model(inputs=model.inputs, outputs=output)
# compile model
opt = SGD(lr=1e-4, momentum=0.1)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
bloc

In [ ]:
BATCH_SIZE = 64
train_datagen = ImageDataGenerator(featurewise_center=True)
valid_datagen = ImageDataGenerator(featurewise_center=True)

# specify imagenet mean values for centering
train_datagen.mean = [123.68, 116.779, 103.939]
valid_datagen.mean = [123.68, 116.779, 103.939]

# prepare iterator
train_generator = train_datagen.flow_from_directory(
        './train',
        target_size=(224, 224),
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False
    )

validation_generator = valid_datagen.flow_from_directory(
        './valid',
        target_size=(224, 224),
        batch_size=BATCH_SIZE,
        class_mode='binary'
    )

# fit model
train_history = model.fit_generator(
        train_generator,
        epochs=20,
        steps_per_epoch = len(train_generator),
        validation_data = validation_generator,
        validation_steps = len(validation_generator),
        verbose = 2
)

# show_train_history
show_train_history(train_history, 'acc', 'val_acc')
show_train_history(train_history, 'loss', 'val_loss')

# save model
model.save('model-VGG16.h5')

Found 3200 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
 - 47s - loss: 0.9717 - acc: 0.9031 - val_loss: 5.5722 - val_acc: 0.5487
Epoch 2/20
 - 38s - loss: 0.6463 - acc: 0.9175 - val_loss: 2.9153 - val_acc: 0.6987
Epoch 3/20
 - 38s - loss: 0.3694 - acc: 0.9387 - val_loss: 1.9160 - val_acc: 0.7712
Epoch 4/20
 - 38s - loss: 0.2532 - acc: 0.9513 - val_loss: 1.3941 - val_acc: 0.8163
Epoch 5/20
 - 39s - loss: 0.1792 - acc: 0.9634 - val_loss: 1.1201 - val_acc: 0.8438
Epoch 6/20
 - 39s - loss: 0.1359 - acc: 0.9700 - val_loss: 0.9157 - val_acc: 0.8700
Epoch 7/20
 - 39s - loss: 0.1064 - acc: 0.9800 - val_loss: 0.7470 - val_acc: 0.8912
Epoch 8/20


In [4]:
# VGG19
from keras.applications.vgg19 import VGG19

# load model
model = VGG19(include_top=False, input_shape=(128, 173, 3))
# mark loaded layers as not trainable
for layer in model.layers:
    layer.trainable = False
# add new classifier layers
flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
output = Dense(1, activation='sigmoid')(class1)
# define new model
model = Model(inputs=model.inputs, outputs=output)
# compile model
opt = SGD(lr=1e-4, momentum=0.1)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 173, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 173, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 173, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 86, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 86, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 86, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 43, 128)       0         
__________

In [ ]:
BATCH_SIZE = 64
train_datagen = ImageDataGenerator(featurewise_center=True)
valid_datagen = ImageDataGenerator(featurewise_center=True)

# specify imagenet mean values for centering
train_datagen.mean = [123.68, 116.779, 103.939]
valid_datagen.mean = [123.68, 116.779, 103.939]

# prepare iterator
train_generator = train_datagen.flow_from_directory(
        './train',
        target_size=(128, 173),
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False
    )

validation_generator = valid_datagen.flow_from_directory(
        './valid',
        target_size=(128, 173),
        batch_size=BATCH_SIZE,
        class_mode='binary'
    )

# fit model
train_history = model.fit_generator(
        train_generator,
        epochs=20,
        steps_per_epoch = len(train_generator),
        validation_data = validation_generator,
        validation_steps = len(validation_generator),
        verbose = 2
)

# show_train_history
show_train_history(train_history, 'acc', 'val_acc')
show_train_history(train_history, 'loss', 'val_loss')

# save model
model.save('model-VGG19.h5')

In [13]:
import os
import glob
# load test data
file_id = []
test_image = []
for f in glob.glob(os.path.join('.\\test','*.jpg')):
    img = image.load_img(f, target_size = (224, 224))    #VGG16
    #img = image.load_img(f, target_size = (128, 173))    #VGG19
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    test_image.append(img)
    file_id.append(f)

test_data = np.array(test_image)
#print(test_data[0:3])

In [14]:
cls_list = ['0', '1']
pred = model.predict(test_image[0])[0]
top_inds = pred.argsort()[::-1][0]
print(top_inds)

pred = model.predict(test_image[1])[0]
top_inds = pred.argsort()[::-1][0]
print(top_inds)

0
0


In [21]:
pred = model.predict(test_image[0])
pred[0][0]

1.5005576e-10

In [22]:
# making predictions
cls_list = ['0', '1']
prediction = []

for img in test_image:
    pred = model.predict(img)[0][0]
    #prediction.append(decode_predictions(pred, top=1)[0])
    #top_inds = pred.argsort()[::-1][0]
    #prediction.append(top_inds)
    prediction.append(pred)

In [23]:
print(np.array(prediction).shape)

(400,)


In [24]:
from datetime import datetime
# creating submission file
submission = pd.DataFrame(columns=['ID', 'Predicted'])
submission['ID'] = file_id
submission['Predicted'] = prediction
submission.to_csv('submission_'+ datetime.now().strftime('%Y%m%d%H%M%S') +'.csv', header=True, index=False)